In [2]:
import pandas as pd

output_file = "US_Disasters_2000_2024.csv"

# Load final merged dataset
df = pd.read_csv(output_file)

# Standardize column names just in case
df.columns = [c.strip().lower() for c in df.columns]

print("Columns:", df.columns.tolist())

# ----------------------------
# 1. Events per year
# ----------------------------
events_per_year = df.groupby("year").size().reset_index(name="event_count")
print("\n=== Events per Year ===")
print(events_per_year)

# ----------------------------
# 2. Events per disaster type
# ----------------------------
events_per_type = df.groupby("disaster_name").size().reset_index(name="event_count")
print("\n=== Events per Disaster Type ===")
print(events_per_type)

# ----------------------------
# 3. Fatalities per year
# ----------------------------
fatalities_per_year = df.groupby("year")["fatalities"].sum().reset_index()
print("\n=== Fatalities per Year ===")
print(fatalities_per_year)

# ----------------------------
# 4. Financial losses per year
# ----------------------------
loss_per_year = df.groupby("year")["loss"].sum().reset_index()
print("\n=== Financial Losses per Year (in USD) ===")
print(loss_per_year)

# OPTIONAL: Save summary tables
events_per_year.to_csv("stats_events_per_year.csv", index=False)
events_per_type.to_csv("stats_events_per_type.csv", index=False)
fatalities_per_year.to_csv("stats_fatalities_per_year.csv", index=False)
loss_per_year.to_csv("stats_losses_per_year.csv", index=False)

Columns: ['year', 'state', 'disaster_name', 'fatalities', 'loss', 'source']

=== Events per Year ===
    year  event_count
0   2000        52007
1   2001        48875
2   2002        50936
3   2003        52956
4   2004        52409
5   2005        53976
6   2006        56400
7   2007        59011
8   2008        71190
9   2009        57398
10  2010        62807
11  2011        79091
12  2012        64503
13  2013        59986
14  2014        59475
15  2015        57907
16  2016        56005
17  2017        57029
18  2018        62697
19  2019        67861
20  2020        61278
21  2021        61389
22  2022        69887
23  2023        75593
24  2024        69493

=== Events per Disaster Type ===
                 disaster_name  event_count
0        Astronomical Low Tide          655
1                    Avalanche          771
2                     Blizzard        13464
3                Coastal Flood         3865
4              Cold/Wind Chill        11684
5                  Debris Flo

In [3]:
import pandas as pd
import requests
import io
import re
from us import states

# ------------------------------------------------------------
# NOAA Storm Events
# ------------------------------------------------------------

def find_noaa_file_for_year(year):
    base = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
    html = requests.get(base).text

    pattern = rf"StormEvents_details-ftp_v1\.0_d{year}_c\d+\.csv\.gz"
    matches = re.findall(pattern, html)

    if not matches:
        return None

    return sorted(matches)[-1]  # latest file


def extract_year(df):
    """Extract event year from available columns."""
    if "BEGIN_YEAR" in df.columns:
        return df["BEGIN_YEAR"]

    if "YEAR" in df.columns:
        return df["YEAR"]

    if "BEGIN_DATE_TIME" in df.columns:
        return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.year

    if "BEGIN_DATE" in df.columns:
        return pd.to_datetime(df["BEGIN_DATE"], errors="coerce").dt.year

    return None


def extract_month(df):
    """Extract event month from available NOAA columns."""
    if "BEGIN_MONTH" in df.columns:
        return df["BEGIN_MONTH"]

    if "BEGIN_DATE_TIME" in df.columns:
        return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month

    if "BEGIN_DATE" in df.columns:
        return pd.to_datetime(df["BEGIN_DATE"], errors="coerce").dt.month

    return None


def flexible_col(df, col, default=0):
    """Safe getter: returns column if exists, else default."""
    return df[col] if col in df.columns else default


def parse_money(x):
    """Convert NOAA money strings like '15K', '3.1M' to numeric dollars."""
    if pd.isna(x) or x == "0":
        return 0
    try:
        m = x[-1]
        v = float(x[:-1])
        if m == "K":
            return v * 1_000
        elif m == "M":
            return v * 1_000_000
        else:
            return float(x)
    except:
        return 0


def download_noaa():
    dfs = []

    for year in range(2000, 2025):
        file_name = find_noaa_file_for_year(year)
        if not file_name:
            print(f"No NOAA files found for {year}")
            continue

        url = f"https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/{file_name}"
        print("Downloading NOAA:", url)

        try:
            df = pd.read_csv(io.BytesIO(requests.get(url).content),
                             compression="gzip", low_memory=False)

            # Add year + month
            df["year"] = extract_year(df)
            df["month"] = extract_month(df)

            # Build columns flexibly
            df["fatalities"] = (
                flexible_col(df, "DEATHS_DIRECT") +
                flexible_col(df, "DEATHS_INDIRECT")
            )

            df["loss"] = flexible_col(df, "DAMAGE_PROPERTY").apply(parse_money)

            df["STATE"] = flexible_col(df, "STATE")
            df["disaster_name"] = flexible_col(df, "EVENT_TYPE", default="Unknown")
            df["source"] = "NOAA"

            dfs.append(df[["year", "month", "STATE", "disaster_name",
                           "fatalities", "loss", "source"]])

        except Exception as e:
            print(f"Skipping {year} (error: {e})")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame()


# ------------------------------------------------------------
# FEMA Disaster Declarations API
# ------------------------------------------------------------

def download_fema():
    """Download FEMA disaster declarations since 2000."""
    print("Downloading FEMA…")

    url = (
        "https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries?"
        "$limit=50000"
    )

    data = requests.get(url).json()["DisasterDeclarationsSummaries"]
    df = pd.DataFrame(data)

    # Ensure required columns exist
    required = ["fyDeclared", "state", "incidentType", "declarationTitle"]
    for col in required:
        if col not in df.columns:
            df[col] = None

    # Extract month from incidentBeginDate if available
    if "incidentBeginDate" in df.columns:
        df["month"] = pd.to_datetime(df["incidentBeginDate"], errors="coerce").dt.month
    else:
        df["month"] = None

    df = df[required + ["month"]].copy()

    df = df.rename(columns={
        "fyDeclared": "year",
        "state": "STATE",
        "declarationTitle": "disaster_name"
    })

    df["fatalities"] = None   # FEMA does not include casualties
    df["loss"] = None         # FEMA does not include dollar loss
    df["source"] = "FEMA"

    return df[["year", "month", "STATE", "disaster_name", "fatalities", "loss", "source"]]


# ------------------------------------------------------------
# USGS Earthquakes  (commented out in your script but updated)
# ------------------------------------------------------------

US_STATE_ABBR = {
    "ALABAMA":"AL","ALASKA":"AK","ARIZONA":"AZ","ARKANSAS":"AR","CALIFORNIA":"CA","COLORADO":"CO",
    "CONNECTICUT":"CT","DELAWARE":"DE","FLORIDA":"FL","GEORGIA":"GA","HAWAII":"HI","IDAHO":"ID",
    "ILLINOIS":"IL","INDIANA":"IN","IOWA":"IA","KANSAS":"KS","KENTUCKY":"KY","LOUISIANA":"LA",
    "MAINE":"ME","MARYLAND":"MD","MASSACHUSETTS":"MA","MICHIGAN":"MI","MINNESOTA":"MN",
    "MISSISSIPPI":"MS","MISSOURI":"MO","MONTANA":"MT","NEBRASKA":"NE","NEVADA":"NV",
    "NEW HAMPSHIRE":"NH","NEW JERSEY":"NJ","NEW MEXICO":"NM","NEW YORK":"NY","NORTH CAROLINA":"NC",
    "NORTH DAKOTA":"ND","OHIO":"OH","OKLAHOMA":"OK","OREGON":"OR","PENNSYLVANIA":"PA",
    "RHODE ISLAND":"RI","SOUTH CAROLINA":"SC","SOUTH DAKOTA":"SD","TENNESSEE":"TN","TEXAS":"TX",
    "UTAH":"UT","VERMONT":"VT","VIRGINIA":"VA","WASHINGTON":"WA","WEST VIRGINIA":"WV",
    "WISCONSIN":"WI","WYOMING":"WY"
}

STATE_ABBR_TO_NAME = {v: k for k, v in US_STATE_ABBR.items()}


def extract_us_state(place: str):
    """Extract U.S. state from USGS 'place' text."""
    if not isinstance(place, str):
        return None

    text = place.upper()

    for state in US_STATE_ABBR:
        if state in text:
            return US_STATE_ABBR[state]

    tokens = re.findall(r"[A-Z]{2}", text)
    for tok in tokens:
        if tok in STATE_ABBR_TO_NAME:
            return tok

    return None


# ------------------------------------------------------------
# Merge All Datasets
# ------------------------------------------------------------

def build_master():
    print("\n=== NOAA ===")
    noaa = download_noaa()

    # Uncomment when ready
    # print("\n=== FEMA ===")
    # fema = download_fema()

    # print("\n=== USGS ===")
    # usgs = download_usgs()

    # Normalize NOAA state names (full → abbreviation)
    state_map = {s.name: s.abbr for s in states.STATES}
    if not noaa.empty:
        noaa["STATE"] = noaa["STATE"].replace(state_map)

    combined = pd.concat([
        noaa[["year", "month", "STATE", "disaster_name", "fatalities", "loss", "source"]],
        # fema[["year", "month", "STATE", "disaster_name", "fatalities", "loss", "source"]],
        # usgs
    ], ignore_index=True)

    combined = combined.sort_values(by=["year", "month", "STATE"], na_position="last")

    output_file = "US_Disasters_2000_2024.csv"
    combined.to_csv(output_file, index=False)

    print(f"\nSaved → {output_file}")
    return combined


if __name__ == "__main__":
    df = build_master()
    print(df.head())



=== NOAA ===


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


...

C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month


C:\Users\Madhur\AppData\Local\Temp\ipykernel_25124\3830751287.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(df["BEGIN_DATE_TIME"], errors="coerce").dt.month



Saved → US_Disasters_2000_2024.csv
     year  month    STATE      disaster_name  fatalities     loss source
843  2000      1  ALABAMA  Thunderstorm Wind           0   1000.0   NOAA
844  2000      1  ALABAMA  Thunderstorm Wind           0   1000.0   NOAA
845  2000      1  ALABAMA  Thunderstorm Wind           0  15000.0   NOAA
938  2000      1  ALABAMA               Heat           0      0.0   NOAA
939  2000      1  ALABAMA               Heat           0      0.0   NOAA


In [4]:
df.head()

,year,month,STATE,disaster_name,fatalities,loss,source
843,2000,1,ALABAMA,Thunderstorm Wind,0,1000.0,NOAA
844,2000,1,ALABAMA,Thunderstorm Wind,0,1000.0,NOAA
845,2000,1,ALABAMA,Thunderstorm Wind,0,15000.0,NOAA
938,2000,1,ALABAMA,Heat,0,0.0,NOAA
939,2000,1,ALABAMA,Heat,0,0.0,NOAA


In [12]:
import numpy as np
import pandas as pd
import math

def custom_round(x):
    """
    x: numeric
    If decimal >= 0.2 → ceil
    else → floor
    """
    frac = x - math.floor(x)
    if frac >= 0.2:
        return math.ceil(x)
    return math.floor(x)


def generate_predictions(df):
    """
    df must contain columns:
    ['year', 'month', 'STATE', 'disaster_name', 'fatalities', 'loss']
    """

    df = df.copy().replace([np.inf, -np.inf], np.nan).fillna(0)

    # ---------------------------------------
    # 1. Historical means per state & month
    # ---------------------------------------
    hist = df.groupby(["STATE", "month"]).agg(
        mean_fatalities=("fatalities", "mean"),
        mean_loss=("loss", "mean")
    ).reset_index()

    hist = hist.fillna(0)

    # ---------------------------------------
    # 2. Most likely disaster per state+month
    # ---------------------------------------
    disaster_counts = (
        df.groupby(["STATE", "month", "disaster_name"])
          .size()
          .reset_index(name="count")
    )

    most_likely = disaster_counts.loc[
        disaster_counts.groupby(["STATE", "month"])["count"].idxmax()
    ][["STATE", "month", "disaster_name"]]

    most_likely = most_likely.rename(columns={
        "disaster_name": "most_likely_disaster"
    })

    # ---------------------------------------
    # 3. Generate prediction frame for next year
    # ---------------------------------------
    next_year = df["year"].max() + 1
    all_states = df["STATE"].unique()
    all_months = np.arange(1, 12 + 1)

    pred_base = pd.MultiIndex.from_product(
        [[next_year], all_months, all_states],
        names=["year", "month", "STATE"]
    ).to_frame(index=False)

    # ---------------------------------------
    # 4. Merge
    # ---------------------------------------
    pred = pred_base.merge(hist, on=["STATE", "month"], how="left")
    pred = pred.merge(most_likely, on=["STATE", "month"], how="left")

    pred["mean_fatalities"] = pred["mean_fatalities"].fillna(0)
    pred["mean_loss"] = pred["mean_loss"].fillna(0)
    pred["most_likely_disaster"] = pred["most_likely_disaster"].fillna("None Observed")

    # ---------------------------------------
    # 5. Rename
    # ---------------------------------------
    pred = pred.rename(columns={
        "mean_fatalities": "predicted_fatalities",
        "mean_loss": "predicted_loss"
    })

    pred = pred.replace([np.inf, -np.inf], 0).fillna(0)

    # ---------------------------------------
    # 6. APPLY CUSTOM ROUNDING
    # ---------------------------------------
    pred["predicted_fatalities"] = pred["predicted_fatalities"].apply(custom_round)
    pred["predicted_loss"] = pred["predicted_loss"].apply(custom_round)

    return pred

pred = generate_predictions(df)

# Save prediction CSV
output_file = "US_Disasters_Prediction_2025.csv"
pred.to_csv(output_file, index=False)

print(f"Saved → {output_file}")


Saved → US_Disasters_Prediction_2025.csv


In [10]:
df.head()

,year,month,STATE,disaster_name,fatalities,loss,source
0,2000,1,ALABAMA,Thunderstorm Wind,0,1000.0,NOAA
1,2000,1,ALABAMA,Thunderstorm Wind,0,1000.0,NOAA
2,2000,1,ALABAMA,Thunderstorm Wind,0,15000.0,NOAA
3,2000,1,ALABAMA,Heat,0,0.0,NOAA
4,2000,1,ALABAMA,Heat,0,0.0,NOAA


In [13]:
import pandas as pd
import numpy as np

def analyze_predictions(pred):
    print("\n==========================")
    print("  ⚡ 2025 PREDICTION ANALYTICS")
    print("==========================\n")

    # -----------------------------------------------
    # 1. Total fatalities & loss per state
    # -----------------------------------------------
    totals = pred.groupby("STATE").agg(
        total_fatalities=("predicted_fatalities", "sum"),
        total_loss=("predicted_loss", "sum")
    ).reset_index()

    print("\n--- Total Fatalities & Loss per State ---")
    print(totals.sort_values("total_fatalities", ascending=False).head(10))

    # -----------------------------------------------
    # 2. States with maximum fatalities and loss
    # -----------------------------------------------
    max_fatal_state = totals.loc[totals["total_fatalities"].idxmax()]
    max_loss_state = totals.loc[totals["total_loss"].idxmax()]

    print("\n--- State with Maximum Fatalities ---")
    print(max_fatal_state)

    print("\n--- State with Maximum Loss ---")
    print(max_loss_state)

    # -----------------------------------------------
    # 3. Deadliest month per state
    # -----------------------------------------------
    monthly_fatal = (
        pred.groupby(["STATE", "month"])["predicted_fatalities"]
        .sum()
        .reset_index()
    )

    deadliest_month = monthly_fatal.loc[
        monthly_fatal.groupby("STATE")["predicted_fatalities"].idxmax()
    ]

    print("\n--- Deadliest Month per State ---")
    print(deadliest_month.head(10))

    # -----------------------------------------------
    # 4. Most costly month per state
    # -----------------------------------------------
    monthly_loss = (
        pred.groupby(["STATE", "month"])["predicted_loss"]
        .sum()
        .reset_index()
    )

    costliest_month = monthly_loss.loc[
        monthly_loss.groupby("STATE")["predicted_loss"].idxmax()
    ]

    print("\n--- Most Expensive Month per State ---")
    print(costliest_month.head(10))

    # -----------------------------------------------
    # 5. Nationwide totals
    # -----------------------------------------------
    nat_fatal = pred["predicted_fatalities"].sum()
    nat_loss = pred["predicted_loss"].sum()

    print("\n--- Nationwide Totals ---")
    print(f"Total predicted fatalities nationwide: {nat_fatal}")
    print(f"Total predicted losses nationwide:    ${nat_loss:,}")

    # -----------------------------------------------
    # 6. Correlation between fatalities & loss
    # -----------------------------------------------
    corr = pred["predicted_fatalities"].corr(pred["predicted_loss"])

    print("\n--- Correlation Fatalities vs Loss ---")
    print(f"Correlation: {corr:.3f}")

    # -----------------------------------------------
    # 7. Return all tables for further use
    # -----------------------------------------------
    return {
        "totals": totals,
        "deadliest_month": deadliest_month,
        "costliest_month": costliest_month,
        "nationwide_totals": (nat_fatal, nat_loss),
        "correlation": corr
    }


In [14]:
analyze_predictions(pred)


  ⚡ 2025 PREDICTION ANALYTICS


--- Total Fatalities & Loss per State ---
             STATE  total_fatalities  total_loss
15            GUAM                 7    49749018
16     GUAM WATERS                 4           0
12       E PACIFIC                 3        8592
43          NEVADA                 3      688615
3          ARIZONA                 2     2445090
20   HAWAII WATERS                 2           0
33       LOUISIANA                 1    15662322
52          OREGON                 1     1945993
2   AMERICAN SAMOA                 1     5924654
63  VIRGIN ISLANDS                 1      269003

--- State with Maximum Fatalities ---
STATE                   GUAM
total_fatalities           7
total_loss          49749018
Name: 15, dtype: object

--- State with Maximum Loss ---
STATE                   GUAM
total_fatalities           7
total_loss          49749018
Name: 15, dtype: object

--- Deadliest Month per State ---
              STATE  month  predicted_fatalities
0       

{'totals':              STATE  total_fatalities  total_loss
 0          ALABAMA                 0     1116835
 1           ALASKA                 0     1533308
 2   AMERICAN SAMOA                 1     5924654
 3          ARIZONA                 2     2445090
 4         ARKANSAS                 0     1695480
 ..             ...               ...         ...
 64        VIRGINIA                 0      623667
 65      WASHINGTON                 0     8418373
 66   WEST VIRGINIA                 0      642890
 67       WISCONSIN                 0      765194
 68         WYOMING                 0      196053
 
 [69 rows x 3 columns],
 'deadliest_month':               STATE  month  predicted_fatalities
 0           ALABAMA      1                     0
 12           ALASKA      1                     0
 32   AMERICAN SAMOA      9                     1
 41          ARIZONA      6                     1
 48         ARKANSAS      1                     0
 ..              ...    ...                  